In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from tezcatli_scripts import load_data, utils, pre_process , write_to_database as w2d #, get_ts_features , fit_models,
from tezcatli_scripts.fit_models import  Darts, Orbit

### For tracking execution times
from os import path
import time
#import random
from random import getrandbits, seed
import pickle

#For reading options from command line
import sys, getopt

import warnings
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20,6]
pd.set_option('display.max_columns',12)
pd.set_option('display.width', 1000)

/home/shahid/anaconda3/envs/jameshardie/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from darts.models import Prophet
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    NaiveMean,
    ExponentialSmoothing,
    AutoARIMA,
    #StandardRegressionModel,
    Theta,
    FFT,
    Croston,
    LightGBMModel,
    RandomForest,
    RegressionEnsembleModel,
    TBATS,
    BATS,
    RegressionModel)
from darts.metrics import mape, smape, mase
from orbit.models import DLT


starting of paralle.py
paralle.py/parallel is printing


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpr_ajnrvy
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpr_ajnrvy/_remote_module_non_scriptable.py


In [4]:
#%% Models to run ###
model_frames = ['Darts','Orbit']
dart_models = [ExponentialSmoothing(), NaiveSeasonal(), NaiveDrift(), NaiveMean(), AutoARIMA(), Theta(), FFT(), Prophet(),Croston(),LightGBMModel(lags=1),RandomForest(lags= 1,random_state=2309),RegressionEnsembleModel(forecasting_models=[ExponentialSmoothing(), NaiveSeasonal(),AutoARIMA(),TBATS()],regression_train_n_points=24),TBATS(),BATS(),RegressionModel(lags=1),RegressionModel(lags=10)]#, StandardRegressionModel()]
dart_models_names = ['ExponentialSmoothing', 'NaiveSeasonal','NaiveDrift','NaiveMean','AutoARIMA','Theta', 'FFT','Prophet','Croston','LightGBMModel','RandomForest','RegressionEnsembleModel','TBATS','BATS','RegressionModelL1','RegressionModelL10']#, 'StandardRegression']
models = [DLT(),ExponentialSmoothing(), NaiveSeasonal(), NaiveDrift(), NaiveMean(), AutoARIMA()]#, Theta(), FFT(), Prophet(),Croston(),LightGBMModel(lags=1),RandomForest(lags= 1,random_state=2309),RegressionEnsembleModel(forecasting_models=[ExponentialSmoothing(), NaiveSeasonal(),AutoARIMA(),TBATS()],regression_train_n_points=24),TBATS(),BATS(),RegressionModel(lags=1),RegressionModel(lags=10)]#, StandardRegressionModel()]
models_names = ['Orbit','ExponentialSmoothing', 'NaiveSeasonal','NaiveDrift','NaiveMean','AutoARIMA']#,'Theta', 'FFT','Prophet','Croston','LightGBMModel','RandomForest','RegressionEnsembleModel','TBATS','BATS','RegressionModelL1','RegressionModelL10']#, 'StandardRegression']
orbit_models = [DLT()]
orbit_models_names = ['DampedLinearTrend']

In [5]:
run_config = utils.read_params_in_from_json('run_config.json')
group_key = run_config['dimensions'].split('-')
#seed(2309)
print(group_key)
run_id = getrandbits(32)
run_date = dt.datetime.today().date()
run_datascientist = run_config['data_scientist']
run_scope = run_config['scope']
run_response = run_config['response']
run_timegrain = run_config['timegrain']
run_dimensions = run_config['dimensions']
run_type = run_config['type']
holdout_horizon = run_config['train_horizon']
forecast_horizon = run_config['forecast_horizon']

test_run = True

write_file = 'yes'

['Region_Name', 'forecast_group']


In [6]:
print('run_id',run_id)
print('run_date',run_date)
print('run_datascientist',run_datascientist)
print('holdout_horizon',holdout_horizon)
print('forecast_horizon',forecast_horizon)

run_id 2569319578
run_date 2023-08-17
run_datascientist shahid_azad
holdout_horizon 5
forecast_horizon 24


In [8]:
prep_comp_prod = pd.read_pickle('prep_init.pkl')
prep_comp_prod.tail()

,Order_Create_Date,Order_Volume_(STD),group_key,run_id
48936,2023-12-01,4046.4,Southwest_C+ Ma HLD,2688014460
48937,2024-01-01,369.6,Southwest_C+ Ma HLD,2688014460
48938,2024-02-01,552.8,Southwest_C+ Ma HLD,2688014460
48939,2024-03-01,929.6,Southwest_C+ Ma HLD,2688014460
48940,2024-04-01,3646.4,Southwest_C+ Ma HLD,2688014460


In [11]:
t_train_start = time.time()
prod_dfs, prod_accs,failed_keys = [],[],[]
#ts_feats = []
## Certain stat models have constraints on length of time series , see later checks##
time_models = ['ExponentialSmoothing']
time_models2 = ['AutoARIMA']
time_models3 = ['RegressionEnsembleModel','TBATS','BATS']
cnt = 0
keys = prep_comp_prod['group_key'].unique()

In [12]:
#key = np.where(keys=='Midwest Central_C+ St Plk')
#keys[key].item()
key = 'Midwest Central_C+ St Plk'

In [13]:
# filter data
prod_df = prep_comp_prod[prep_comp_prod['group_key']==key]
prod_df

,Order_Create_Date,Order_Volume_(STD),group_key,run_id
10246,2015-04-01,1967361.25,Midwest Central_C+ St Plk,2688014460
10247,2015-05-01,1916091.00,Midwest Central_C+ St Plk,2688014460
10248,2015-06-01,2063669.25,Midwest Central_C+ St Plk,2688014460
10249,2015-07-01,1831827.50,Midwest Central_C+ St Plk,2688014460
10250,2015-08-01,1311524.75,Midwest Central_C+ St Plk,2688014460
...,...,...,...,...
10350,2023-12-01,1747686.75,Midwest Central_C+ St Plk,2688014460
10351,2024-01-01,1534704.00,Midwest Central_C+ St Plk,2688014460
10352,2024-02-01,2193432.25,Midwest Central_C+ St Plk,2688014460
10353,2024-03-01,2478356.75,Midwest Central_C+ St Plk,2688014460


In [14]:
models_list = list(zip(models,models_names))
model_dfs, model_accs = [],[]

In [15]:
models_list

[(<orbit.forecaster.full_bayes.FullBayesianForecaster at 0x7fb7d1cc4e90>,
  'Orbit'),
 (ExponentialSmoothing(trend=ModelMode.ADDITIVE, damped=False, seasonal=SeasonalityMode.ADDITIVE, seasonal_periods=None, random_state=0),
  'ExponentialSmoothing'),
 (NaiveSeasonal(K=1), 'NaiveSeasonal'),
 (NaiveDrift(), 'NaiveDrift'),
 (NaiveMean(), 'NaiveMean'),
 (AutoARIMA(add_encoders=None), 'AutoARIMA')]

In [16]:
run_mofcst = dt.datetime(run_config['current_year'],run_config['current_month'],1)
train_date = run_mofcst + relativedelta(months=-holdout_horizon)

In [23]:
print('train_date',train_date)
print('run_mofcst',run_mofcst)

train_date 2023-12-01 00:00:00
run_mofcst 2024-05-01 00:00:00


In [17]:
log_file_path = path.join(path.dirname(path.abspath('params/log.conf')), 'log.conf')
jhds_logger = utils.setup_logger(log_file_path)
jhds_logger.info('Finished setup')

In [18]:
start_time = time.perf_counter()
for model,name in list(zip(models,models_names)):
    #### Instantiate model framework
    if (name == 'Orbit'):
        model_frame = Orbit(model,prod_df,run_mofcst,train_date=train_date,forecast_horizon=None)
    else:
        model_frame = Darts(model,prod_df, run_mofcst, train_date,forecast_horizon=None)
    #print(model_frame)

    #### Create time series
    #prod_ts = pre_process.create_ts(prod_df,run_mofcst)
    model_frame.prep_data()

    #### Split train and test sets for holdout accuracy
    #train,val = prod_ts.split_before(pd.Timestamp(train_date))
    model_frame.split_data()
    #TODO create fit function (returns model params)
    # check for length , Expo can't handle less than 24, autoarima needs 30
    if ( (len(model_frame.train)<24) & (name in time_models) ):
        failed_keys.append((key,name))
        continue
    elif ( (len(model_frame.train)<30) & (name in time_models2)) :
        failed_keys.append((key,name))
        continue
    elif ( (name in time_models3) & (model_frame.get_train_df_ordervol().tail(12).sum()<12) ):
        failed_keys.append((key,name))
        continue
    #### Fit and pred ##
    t_modelfitpred_start = time.time()
    try:
        model_frame.train_model()
        #model.fit(train)
    except ZeroDivisionError:
        jhds_logger.error(f'A zero division error occurred with key {key} in training model {name}')

    try:
        pred = model_frame.pred_model()
        #pred = model.predict(len(val))
    except ValueError:
        jhds_logger.error(f'Training model {name} with key {key} failed due to NaN, infinity or too large number')

    pred_df = pred.pd_dataframe()
    pred_df['model'] = name
    pred_df.rename(columns={'0':'fcst'},inplace=True)
    model_dfs.append(pred_df)
    #### Accuracies ##
    accuracies = pd.DataFrame()
    accuracies['mape'] = pd.Series(mape(model_frame.val,pred))
    accuracies['smape'] = pd.Series(smape(model_frame.val,pred))
    accuracies['mase'] = pd.Series(mase(model_frame.val,pred,insample=model_frame.train))
    accuracies['model'] = name
    model_accs.append(accuracies)
    t_modelfitpred_end = time.time()
    jhds_logger.info(f'Finished fitting model {name}, it took {"{:.2f}".format((t_modelfitpred_end-t_modelfitpred_start)/60)} minutes')
finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")
print(model_dfs, model_accs)

model is training now fit_model/Orbit/train_model <orbit.forecaster.full_bayes.FullBayesianForecaster object at 0x7fb7d11c1750>


parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model ExponentialSmoothing()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveSeasonal()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveDrift()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveMean()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model AutoARIMA()


/home/shahid/anaconda3/envs/jameshardie/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,


parallel/delayed is printing
parallel/delayed is printing
Program finished in 3.034291146999749 seconds
[component          Order_Volume_(STD)  model
Order_Create_Date                           
2023-12-01               2.451679e+06  Orbit
2024-01-01               2.469601e+06  Orbit
2024-02-01               2.350747e+06  Orbit
2024-03-01               2.477421e+06  Orbit
2024-04-01               2.107309e+06  Orbit, component          Order_Volume_(STD)                 model
Order_Create_Date                                          
2023-12-01               2.594438e+06  ExponentialSmoothing
2024-01-01               2.314034e+06  ExponentialSmoothing
2024-02-01               2.339418e+06  ExponentialSmoothing
2024-03-01               2.456159e+06  ExponentialSmoothing
2024-04-01               2.204377e+06  ExponentialSmoothing, component          Order_Volume_(STD)          model
Order_Create_Date                                   
2023-12-01                 1313500.75  NaiveSeasonal

In [19]:
keys = ['Midwest Central_C+ St Plk', 'Midwest Central_Pr Plk']

In [22]:
# with open('keys.pkl', 'rb') as f:
#     keys = pickle.load(f)

In [20]:
keys

['Midwest Central_C+ St Plk', 'Midwest Central_Pr Plk']

In [17]:
list(zip(models,models_names))

[(<orbit.forecaster.full_bayes.FullBayesianForecaster at 0x7f3f3e67e350>,
  'Orbit'),
 (ExponentialSmoothing(trend=ModelMode.ADDITIVE, damped=False, seasonal=SeasonalityMode.ADDITIVE, seasonal_periods=None, random_state=0),
  'ExponentialSmoothing'),
 (NaiveSeasonal(K=1), 'NaiveSeasonal'),
 (NaiveDrift(), 'NaiveDrift'),
 (NaiveMean(), 'NaiveMean'),
 (AutoARIMA(add_encoders=None), 'AutoARIMA')]

In [21]:
start_time = time.perf_counter()
for key in keys :
    cnt +=1
    len_keys = len(keys)
    jhds_logger.info(f'Working on item : {key}, key {cnt} out of {len_keys} ')
    prod_df = prep_comp_prod[prep_comp_prod['group_key']==key]

    #### Cycle through models ###
    t_model_start = time.time()
    #jhds_logger.info(f'Cycling through models for {key} ')
    model_dfs, model_accs = [],[]
    seed(2309)
    for model,name in list(zip(models,models_names)):
        #### Instantiate model framework
        if (name == 'Orbit'):
            model_frame = Orbit(model,prod_df,run_mofcst,train_date=train_date,forecast_horizon=None)
        else:
            model_frame = Darts(model,prod_df, run_mofcst, train_date,forecast_horizon=None)
        #print(model_frame)

        #### Create time series
        #prod_ts = pre_process.create_ts(prod_df,run_mofcst)
        model_frame.prep_data()

        #### Split train and test sets for holdout accuracy
        #train,val = prod_ts.split_before(pd.Timestamp(train_date))
        model_frame.split_data()
        #TODO create fit function (returns model params)
        # check for length , Expo can't handle less than 24, autoarima needs 30
        if ( (len(model_frame.train)<24) & (name in time_models) ):
            failed_keys.append((key,name))
            continue
        elif ( (len(model_frame.train)<30) & (name in time_models2)) :
            failed_keys.append((key,name))
            continue
        elif ( (name in time_models3) & (model_frame.get_train_df_ordervol().tail(12).sum()<12) ):
            failed_keys.append((key,name))
            continue
        #### Fit and pred ##
        t_modelfitpred_start = time.time()
        try:
            model_frame.train_model()
            #model.fit(train)
        except ZeroDivisionError:
            jhds_logger.error(f'A zero division error occurred with key {key} in training model {name}')

        try:
            pred = model_frame.pred_model()
            #pred = model.predict(len(val))
        except ValueError:
            jhds_logger.error(f'Training model {name} with key {key} failed due to NaN, infinity or too large number')

        pred_df = pred.pd_dataframe()
        pred_df['model'] = name
        pred_df.rename(columns={'0':'fcst'},inplace=True)
        model_dfs.append(pred_df)
        #### Accuracies ##
        accuracies = pd.DataFrame()
        accuracies['mape'] = pd.Series(mape(model_frame.val,pred))
        accuracies['smape'] = pd.Series(smape(model_frame.val,pred))
        accuracies['mase'] = pd.Series(mase(model_frame.val,pred,insample=model_frame.train))
        accuracies['model'] = name
        model_accs.append(accuracies)
        t_modelfitpred_end = time.time()
        jhds_logger.info(f'Finished fitting model {name}, it took {"{:.2f}".format((t_modelfitpred_end-t_modelfitpred_start)/60)} minutes')

    accs_df = pd.concat(model_accs)
    accs_df['group_key'] = key
    models_df = pd.concat(model_dfs)
    models_df['group_key'] = key
    prod_dfs.append(models_df)
    prod_accs.append(accs_df)
    #t_inner.toc('Product Model fit and preds took ')
    t_model_end = time.time()
    jhds_logger.info(f'Finished cycling through models for key: {key}, cycle took {"{:.2f}".format((t_model_end-t_model_start)/60)} minutes')

accs_list_file = 'accs_list.pkl'
preds_list_file = 'preds_list.pkl'
with open(accs_list_file,'wb') as f:
    pickle.dump(prod_accs,f)
with open(preds_list_file,'wb') as f:
    pickle.dump(prod_dfs,f)
jhds_logger.info('Pickled accuracies and predictions in holdout')
prods_mods_df = pd.concat(prod_dfs).reset_index()
prods_accs_df = pd.concat(prod_accs)

## Failed logging ##
failed_df = pd.DataFrame({'group_key':failed_keys})

###
holdout_accs = prods_accs_df.set_index(['model','group_key']).reset_index()
failed_accuracy = holdout_accs[holdout_accs['smape'].isnull()][['group_key','model']]
failed_accuracy['fail_point'] = 'accuracy_calc'
failed_df = pd.concat([failed_df,failed_accuracy])
holdout_accs = holdout_accs.dropna(subset=['smape'])
finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")

model is training now fit_model/Orbit/train_model <orbit.forecaster.full_bayes.FullBayesianForecaster object at 0x7fb7d11c83d0>
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model ExponentialSmoothing()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveSeasonal()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveDrift()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveMean()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model AutoARIMA()


/home/shahid/anaconda3/envs/jameshardie/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,


parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Orbit/train_model <orbit.forecaster.full_bayes.FullBayesianForecaster object at 0x7fb7d11924d0>
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model ExponentialSmoothing()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveSeasonal()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveDrift()
parallel/delayed is printing
parallel/delayed is printing
model is training now fit_model/Darts/train_model NaiveMean()
parallel/delayed is printing
parallel/delayed is printing


/home/shahid/anaconda3/envs/jameshardie/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,


model is training now fit_model/Darts/train_model AutoARIMA()
parallel/delayed is printing
parallel/delayed is printing
Program finished in 6.232601464999789 seconds


In [24]:
prod_dfs

[component          Order_Volume_(STD)                 model                  group_key
 Order_Create_Date                                                                     
 2023-12-01               2.451679e+06                 Orbit  Midwest Central_C+ St Plk
 2024-01-01               2.469601e+06                 Orbit  Midwest Central_C+ St Plk
 2024-02-01               2.350747e+06                 Orbit  Midwest Central_C+ St Plk
 2024-03-01               2.477421e+06                 Orbit  Midwest Central_C+ St Plk
 2024-04-01               2.107309e+06                 Orbit  Midwest Central_C+ St Plk
 2023-12-01               2.594438e+06  ExponentialSmoothing  Midwest Central_C+ St Plk
 2024-01-01               2.314034e+06  ExponentialSmoothing  Midwest Central_C+ St Plk
 2024-02-01               2.339418e+06  ExponentialSmoothing  Midwest Central_C+ St Plk
 2024-03-01               2.456159e+06  ExponentialSmoothing  Midwest Central_C+ St Plk
 2024-04-01               2.2043